In [1]:
import datetime
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Всем привет! Здесь описано подключение к БД, примеры запросов и тд.

### Обратите внимание на признак Probability - его использование зависит от выбранного вами направления. 
### Просим в явном виде отображать его исключение в 1 направлении, мы будем каждый ноутбук проверять и прогонять модели руками (просьба не мухлевать и не использовать информацию из будущего в признаках - мы все заметим :) )

## Формат выгрузки csv-файла (столбцы):
### Для чек-поинтов - это на тренировочных данных, для финального резульата - на тестовых данных

1.   LocationId
2.   ProductId
3.   ValidationDateTime
4.   CalculatedProbability (вычисленная вами вероятность - predict_proba() )

### Также будет очень приятно, если вы нам выведите feature_importances в виде картиночки :)

In [39]:
conn_str = """
    Driver={ODBC Driver 17 for SQL Server};
    Server=109.71.11.235;
    Database=GFC.Hackathon;
    UID=participant;
    PWD=AiNRYmEatiBL;
"""

conn = pyodbc.connect(conn_str)

In [38]:
# пример запросов - здесь выведен весь датасет признаков
query_features = "select * from dbo.Features"
df = pd.read_sql(query_features, conn)
df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288527,2023-06-28 14:57:07,0.416474,False,86.0,85.0,0.011628,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
1,63,288606,2023-06-01 15:32:31,0.687710,False,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
2,63,288606,2023-06-03 12:55:49,0.810382,False,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
3,63,288606,2023-06-07 14:57:38,0.312500,False,4.0,3.0,0.250000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
4,63,288606,2023-06-09 14:56:35,0.605429,False,6.0,5.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176613,2950,576631,2023-07-12 15:17:30,0.542776,True,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176614,2950,576631,2023-07-15 15:02:34,0.497054,True,4.0,4.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176615,2950,576631,2023-07-26 15:01:27,0.930926,None,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176616,2950,576631,2023-07-28 15:04:19,0.704421,None,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0


In [ ]:
df['ValidationDateTime'] = pd.to_datetime(df['ValidationDateTime'])

In [33]:
# для обучения модели используем записи с IsCorrect is not null
train_query = "select * from dbo.Features f where f.IsCorrect is not null"
train_df = pd.read_sql(train_query, conn)
train_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288527,2023-06-28 14:57:07,0.416474,False,86.0,85.0,0.011628,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
1,63,288606,2023-06-01 15:32:31,0.687710,False,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
2,63,288606,2023-06-03 12:55:49,0.810382,False,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
3,63,288606,2023-06-07 14:57:38,0.312500,False,4.0,3.0,0.250000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
4,63,288606,2023-06-09 14:56:35,0.605429,False,6.0,5.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135842,2950,574768,2023-07-19 15:01:04,0.656481,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135843,2950,575450,2023-06-20 15:00:06,0.778028,False,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135844,2950,576459,2023-07-02 15:01:13,0.248685,False,41.0,40.0,0.024390,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135845,2950,576631,2023-07-12 15:17:30,0.542776,True,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0


In [35]:
# тестовая выборка (то, что используется в финальном зачете) - записи с IsCorrect is null
test_query = "select * from dbo.Features f where f.IsCorrect is null"
test_df = pd.read_sql(test_query, conn)
test_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288606,2023-07-25 14:57:04,0.406708,None,14.0,13.0,0.071429,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
1,63,288606,2023-08-03 14:58:53,0.460643,None,2.0,1.0,0.500000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
2,63,288783,2023-07-24 14:57:11,0.404959,None,4.0,4.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
3,63,288783,2023-08-03 14:57:39,0.659497,None,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
4,63,289136,2023-07-24 14:57:09,0.299769,None,6.0,6.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,2950,572875,2023-07-26 15:01:26,0.248685,None,19.0,18.0,0.052632,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40767,2950,574768,2023-07-31 15:07:00,0.173554,None,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40768,2950,576631,2023-07-26 15:01:27,0.930926,None,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40769,2950,576631,2023-07-28 15:04:19,0.704421,None,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0


In [37]:
train_df.sort_values('ValidationDateTime', inplace=True)
test_df.sort_values('ValidationDateTime', inplace=True)

In [43]:
# дополнительные таблицы с почасовыми продажами и остатками - мы ждем отвас новых интересных признаков :)
sales_query = "select top 100 * from dbo.LocationStateHourSales" 
sales = pd.read_sql(sales_query, conn)
sales

,LocationId,ProductId,Datetime,Quantity,PriceSum,Date
0,63,288606,2023-05-01 12:00:00,6.000,179.940002,2023-05-01
1,63,288606,2023-05-01 13:00:00,2.000,59.980000,2023-05-01
2,63,288783,2023-05-01 17:00:00,2.000,259.799988,2023-05-01
3,63,288913,2023-05-01 16:00:00,1.000,129.899994,2023-05-01
4,63,289117,2023-05-01 11:00:00,3.000,92.970001,2023-05-01
...,...,...,...,...,...,...
95,63,308898,2023-05-01 13:00:00,1.000,44.990002,2023-05-01
96,63,308898,2023-05-01 18:00:00,1.000,22.990000,2023-05-01
97,63,308905,2023-05-01 15:00:00,0.238,78.519997,2023-05-01
98,63,308905,2023-05-01 20:00:00,0.282,93.029999,2023-05-01


In [42]:
stocks_query = "select top 100 * from dbo.LocationStateStocks" 
stocks = pd.read_sql(stocks_query, conn)
stocks

,LocationId,ProductId,Datetime,Quantity,Date
0,63,288524,2023-05-01,21.0,2023-05-01
1,63,288527,2023-05-01,181.0,2023-05-01
2,63,288606,2023-05-01,6.0,2023-05-01
3,63,288783,2023-05-01,11.0,2023-05-01
4,63,288817,2023-05-01,18.0,2023-05-01
...,...,...,...,...,...
95,63,297308,2023-05-01,18.0,2023-05-01
96,63,297309,2023-05-01,18.0,2023-05-01
97,63,297310,2023-05-01,15.0,2023-05-01
98,63,297311,2023-05-01,23.0,2023-05-01


In [45]:
# csv-файл для промежуточных результатов должен быть за следующие даты
train_checkpoint = train_df.query('ValidationDateTime >= "2023-07-15"')
train_checkpoint

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
107777,2648,401972,2023-07-15 14:54:39.167,0.977865,False,0.0,-2.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
107773,2648,401971,2023-07-15 14:55:00.033,0.966208,False,12.0,10.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
661,63,510528,2023-07-15 14:58:10.000,-1.000000,True,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
66920,1719,523365,2023-07-15 14:58:10.000,0.100000,True,6.0,5.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
66605,1719,355836,2023-07-15 14:58:10.000,0.416800,False,0.0,0.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81151,2056,489873,2023-07-21 16:46:15.510,0.977900,True,2.0,1.0,0.500000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
80402,2022,440169,2023-07-21 16:48:18.217,0.973120,True,3.0,3.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
68471,1788,426699,2023-07-21 16:48:32.930,0.972101,True,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
68475,1788,427524,2023-07-21 16:50:45.543,0.975000,False,5.0,4.0,0.200000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
